# Análisis de resultados de Pron vs. Prompt

En este cuaderno analizamos los datos del experimento Pron vs. Prompt. Principalmente, haremos un análisis descriptivo y de correlaciones. Dejamos el modelo de efectos mixtos para R.

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import copy
import numpy as np
import matplotlib

# Configurar la fuente de toda la figura en Times New Roman
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'

Unas variables auxiliares que nos permitirán seleccionar subconjuntos de los datos.

In [49]:
columns_study = ['1_atractivo_titulo',
       '1_atractivo_estilo', '1_atractivo_tema', '2_originalidad_titulo',
       '2_originalidad_estilo', '2_originalidad_tema', '3_relevancia',
       '4_creatividad_titulo', '4_creatividad_sinopsis',
       '5_autoria_titulo', '5_autoria_sinopsis',
       '6_antologia', '6_opinion_lectores',
       '6_opinion_criticos', '6_voz_propia',]
columns_numeric_title = ['1_atractivo_titulo', 
       '2_originalidad_titulo', 
       '4_creatividad_titulo',]
columns_numeric_synopsis = ['1_atractivo_estilo', '1_atractivo_tema',
        '2_originalidad_estilo', '2_originalidad_tema',
       '4_creatividad_sinopsis', ]
       # '6_antologia', '6_opinion_lectores', '6_opinion_criticos', '6_voz_propia']
columns_numeric_synopsis_without6 = ['1_atractivo_estilo', '1_atractivo_tema',
        '2_originalidad_estilo', '2_originalidad_tema',
       '4_creatividad_sinopsis',]

#### Preparación de los DataFrames

Vamos a generar unas ids para las distintas operaciones sobre los datos.
- ``id_item``: nos permite identificar de manera unívoca la sinopsis que ha escrito un agente, pero no distingue entre evaluadores.
- ``id_user``: nos permite identificar cada sinopsis votada por cada crítico. No distingue entre escritores, esto nos permite hacer merge con los datos de Patricio, por un lado, y de GPT por otro.

In [50]:
df = pd.read_csv('data/resultados_en_bruto.csv')
df['id_title_sinopsis_writer'] = df['title_id'] + '_' + df['sinopsis_writer']
df['id_title_title_writer'] = df['title_id'] + '_' + df['title_writer']
df['id_for_es_en'] = df['title_id'] + '_' + df['sinopsis_writer'].replace('_es','').replace('_en','')
df['id_title_user'] = df['title_id'] + '_' + df['username'] 
df.columns

Index(['timestamp', 'username', '1_atractivo_titulo', '1_atractivo_estilo',
       '1_atractivo_tema', '2_originalidad_titulo', '2_originalidad_estilo',
       '2_originalidad_tema', '3_relevancia', '4_creatividad_titulo',
       '4_creatividad_sinopsis', '4_creatividad_comentario',
       '5_autoria_titulo', '5_autoria_sinopsis', '5_autoria_comentario',
       '6_antologia', '6_opinion_lectores', '6_opinion_criticos',
       '6_voz_propia', '6_comentario', 'title', 'title_id', 'sinopsis_writer',
       'title_writer', 'experiment', 'id_title_sinopsis_writer',
       'id_title_title_writer', 'id_for_es_en', 'id_title_user'],
      dtype='object')

Ahora creamos los DataFrame para estudiar, separados por escritor de cada sinopsis, que es el objetivo principal de nuestro experimento. Esa diferencia.

In [51]:
def select_df_by(df, selector = 'sinopsis'):
    df = copy.copy(df)
    df[columns_numeric_synopsis] += 1
    mode = f'{selector}_writer'

    if selector == 'sinopsis':
        df = df.drop(columns=columns_numeric_title).reset_index(drop=True)
        df_gpt_en = df[df[mode] == 'gpt4_en'].reset_index(drop=True)
        df_gpt_es = df[df[mode] == 'gpt4_es'].reset_index(drop=True)
        df_gpt_agrr = df[df[mode] != 'patricio'].reset_index(drop=True)
        join_id = 'id_title_user'
    else:
        df = df.drop(columns=columns_numeric_synopsis).reset_index(drop=True)
        df_gpt_en = df[df[mode] == 'machine'].reset_index(drop=True)
        df_gpt_es = df_gpt_en
        df_gpt_agrr = df[df[mode] != 'patricio'].reset_index(drop=True)
        join_id = 'id_title_user'

    df_es = df[df.experiment=='SPANISH']
    df_en = df[df.experiment=='ENGLISH']

    df_patricio = df[df[mode] == 'patricio'].reset_index(drop=True)
    df_patricio_es = df_es[df_es[mode] == 'patricio']
    df_patricio_en = df_en[df_en[mode] == 'patricio']
    df_patricio_gpt_es = df_patricio_es.merge(df_gpt_es, on=join_id, suffixes=('_patricio', '_gpt4')).transpose().sort_index().transpose()
    df_patricio_gpt_en = df_patricio_en.merge(df_gpt_en, on=join_id, suffixes=('_patricio', '_gpt4')).transpose().sort_index().transpose()
    df_patricio_es_patricio_en = df_patricio_en.merge(df_patricio_es, on='title_id', suffixes=('_patricio_en', '_patricio_es')).transpose().sort_index().transpose()
    
    df_gpt_es_gpt_en = df_gpt_es.merge(df_gpt_en, on='title_id', suffixes=('_gpt4_es', '_gpt4_en')).transpose().sort_index().transpose()

    df_gpt_en.to_csv('data/df_gpt_en.csv')
    df_gpt_es.to_csv('data/df_gpt_es.csv')
    df_patricio.to_csv('data/df_patricio.csv')
    df_patricio_gpt_es.to_csv('data/df_patricio_gpt_es.csv')
    df_patricio_gpt_en.to_csv('data/df_patricio_gpt_en.csv')
    df_patricio_es_patricio_en.to_csv('data/df_patricio_es_patricio_en.csv')
    df_gpt_es_gpt_en.to_csv('data/df_gpt_es_gpt_en.csv')

    return {'df_gpt_en':df_gpt_en,
            'df_gpt_es':df_gpt_es,
            'df_patricio':df_patricio,
            'df_gpt_agrr' : df_gpt_agrr,
            'df_patricio_gpt_es':df_patricio_gpt_es,
            'df_patricio_gpt_en':df_patricio_gpt_en,
            'df_patricio_es_patricio_en':df_patricio_es_patricio_en,
            'df_gpt_es_gpt_en':df_gpt_es_gpt_en}

In [52]:
dfs_title = select_df_by(df, 'title')
dfs_sinopsis = select_df_by(df, 'sinopsis')

In [54]:
# comparación gpt_es vs gpt_en
dfs_sinopsis['df_gpt_es'] = dfs_sinopsis['df_gpt_es'].sort_values(by='title_id').reset_index().add_suffix('_gpt_es')
dfs_sinopsis['df_gpt_en'] = dfs_sinopsis['df_gpt_en'].sort_values(by='title_id').reset_index().add_suffix('_gpt_en')
dfs_sinopsis['df_gpt_es_gpt_en'] = pd.concat([dfs_sinopsis['df_gpt_es'], dfs_sinopsis['df_gpt_en']], axis=1)

## Análisis de significancia

In [57]:
from scipy.stats import wilcoxon

In [58]:
def separate_by_suffixes(string_list, left, rigth):
    suffix1_list = []
    suffix2_list = []
    
    for string in string_list:
        if string.endswith(left):
            suffix1_list.append(string)
        elif string.endswith(rigth):
            suffix2_list.append(string)
    
    return suffix1_list, suffix2_list

In [59]:
def get_wilcoxon_test(data, left = '_gpt4', rigth = '_patricio'):
    
    columns_es, columns_en = separate_by_suffixes(data, left, rigth)
    # Filter numeric columns for both languages
    numeric_columns_es = data[columns_es].select_dtypes(include=[np.number]).columns
    numeric_columns_en = data[columns_en].select_dtypes(include=[np.number]).columns

    # Ensure columns are in the same order for pairing
    numeric_columns_es = sorted([col for col in numeric_columns_es])
    numeric_columns_en = sorted([col for col in numeric_columns_en])

    # Initialize results list
    results = []

    # Perform Wilcoxon test and calculate effect size for each pair
    for col_es, col_en in zip(numeric_columns_es, numeric_columns_en):
        # Drop NaN values to ensure valid comparisons
        paired_data = data[[col_es, col_en]].dropna()
        
        if not paired_data.empty:
            es_values = paired_data[col_es]
            en_values = paired_data[col_en]

            # Perform Wilcoxon signed-rank test
            stat, p_value = wilcoxon(es_values, en_values)

            # Calculate effect size (rank-biserial correlation)
            n = len(paired_data)
            z = stat / np.sqrt(n)
            effect_size = z / np.sqrt(n)

            # Store the results
            results.append({
                'Variable': col_es.replace('_gpt_es', ''),
                'Wilcoxon Statistic': stat,
                'p-value': p_value,
                'Effect Size': effect_size
            })

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Display the results
    return results_df


In [60]:
columns_corr = ["1_atractivo_estilo",
    "1_atractivo_tema",
    "2_originalidad_estilo",
    "2_originalidad_tema",
    "4_creatividad_sinopsis"]

Significación estadística entre GPT4 y Patricio en español.

In [61]:
get_wilcoxon_test(dfs_sinopsis['df_gpt_es_gpt_en'], left = '_gpt_en', rigth = '_gpt_es')

,Variable,Wilcoxon Statistic,p-value,Effect Size
0,1_atractivo_estilo_gpt_en,1636.5,1.205859e-02,9.091667
1,1_atractivo_tema_gpt_en,2974.5,5.610586e-01,16.525000
2,2_originalidad_estilo_gpt_en,1446.0,3.919718e-04,8.033333
3,2_originalidad_tema_gpt_en,2690.0,1.062079e-01,14.944444
4,4_creatividad_sinopsis_gpt_en,2244.5,1.551554e-02,12.469444
5,6_antologia_gpt_en,2047.5,2.376777e-04,11.375000
6,6_opinion_criticos_gpt_en,699.0,3.857861e-16,3.883333
7,6_opinion_lectores_gpt_en,686.0,9.136709e-02,3.811111
8,6_voz_propia_gpt_en,1336.5,3.519064e-08,7.425000
9,index_gpt_en,7308.5,6.477626e-01,40.602778


Para validar la rúbrica, tenemos dos grupos de usuarios que han votado de manera distinta las sinopsis de Patricio. Vamos a comprobar si está bien hecha la rúbrica tratando de observar si hay diferencias entre las distribuciones.